# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#these indeces are region names (type = strings) which we want to remove to calcular
cleanNHL = nhl_df.iloc[0:36]
cleanNHL = cleanNHL.drop(index=[0,9,18,26,35], axis=1)

def nhl_correlation(pci=False): 
    
    #Create a dictionary connecting the population to the team and it's region
    popCityinfo = {}
    for team in cities['NHL']:
        team2 = re.sub('\[note .*]','', team)
        if team2 != '—' and team2!='':
            metropolitanArea = cities['Metropolitan area'].where(cities['NHL'] == team).dropna()
            population = cities['Population (2016 est.)[8]'].where(cities['NHL'] == team).dropna()

            popCityinfo[team2] = [metropolitanArea.values[0], population.values[0]]

    
    #NJ Devils & Kings Ducks is named differently in the other dataframe, so we will rename
    popCityinfo['New Jersey Devils'] = popCityinfo['Rangers Islanders Devils']
    popCityinfo['Anaheim Ducks'] = popCityinfo['Kings Ducks']
    del popCityinfo['Rangers Islanders Devils']
    del popCityinfo['Kings Ducks']
        
    #Create w:l ratio from the other dataframe
    cleanNHL['W:L'] = cleanNHL['W'].astype(int)/(cleanNHL['W'].astype(int)+cleanNHL['L'].astype(int))

    #populate the dictionary with ratio
    for key in popCityinfo:
        for team in cleanNHL['team']:
            if key in team:
                ratio = cleanNHL['W:L'].where(cleanNHL['team'] == team).dropna().values[0]
                popCityinfo[key].append(ratio)  
                
    #Additional teams not mentioned in cities dataframe
    NYC = ['New York Rangers', 'New York Islanders']
    LA = ['Los Angeles Kings*']
    popCityinfo['New Jersey Devils'][-1] += cleanNHL['W:L'].where(cleanNHL['team'] == NYC[0]).dropna().values[0]
    popCityinfo['New Jersey Devils'][-1] += cleanNHL['W:L'].where(cleanNHL['team'] == NYC[1]).dropna().values[0]
    popCityinfo['New Jersey Devils'][-1] = popCityinfo['New Jersey Devils'][-1]/3
    
    popCityinfo['Anaheim Ducks'][-1] += cleanNHL['W:L'].where(cleanNHL['team'] == LA[0]).dropna().values[0]
    popCityinfo['Anaheim Ducks'][-1] = popCityinfo['Anaheim Ducks'][-1]/2
    
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    for item in popCityinfo.items():
        population_by_region.append(int(item[1][1]))
        win_loss_by_region.append(item[1][2])
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    if (pci == True):
        return(popCityinfo)
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.01248616292120991

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#2018 data
cleanNBA = nba_df.iloc[0:30]


def nba_correlation(pci=False):
    popCityinfo = {}
    for team in cities['NBA']:
        team2 = re.sub('\[note .*]','', team)
        if team2 != '—' and team2!='':
            metropolitanArea = cities['Metropolitan area'].where(cities['NBA'] == team).dropna()
            population = cities['Population (2016 est.)[8]'].where(cities['NBA'] == team).dropna()

            popCityinfo[team2] = [metropolitanArea.values[0], population.values[0]]

    #2018 data
    cleanNBA = nba_df.iloc[0:30]

    #Create w:l ratio from the other dataframe
    cleanNBA['W:L'] = cleanNBA['W'].astype(int)/(cleanNBA['W'].astype(int)+cleanNBA['L'].astype(int))

    #populate the dictionary with ratio
    NYC = False
    NYC2 = False
    LA = False
    LA2 = False
    for key in popCityinfo:
        for team in cleanNBA['team']:
            team2 = re.sub('\*','',team)
            team2 = re.sub('\(\d*\)','',team2)
            if key in team2:
                ratio = cleanNBA['W:L'].where(cleanNBA['team'] == team).dropna().values[0]
                popCityinfo[key].append(ratio)
                continue

            if 'New York Knicks' in team2 or 'Brooklyn Nets' in team2:
                if not NYC2:
                    ratio = cleanNBA['W:L'].where(cleanNBA['team'] == team).dropna().values[0]
                    if not NYC:
                        popCityinfo['Knicks Nets'].append(ratio)
                        NYC = True
                    else:
                        popCityinfo['Knicks Nets'][-1] += ratio
                        popCityinfo['Knicks Nets'][-1] = popCityinfo['Knicks Nets'][-1]/2
                        NYC2 = True
            if 'Los Angeles Lakers' in team2 or 'Los Angeles Clippers' in team2:
                if not LA2:
                    ratio = cleanNBA['W:L'].where(cleanNBA['team'] == team).dropna().values[0]
                    if not LA:
                        popCityinfo['Lakers Clippers'].append(ratio)
                        LA = True
                    else:
                        popCityinfo['Lakers Clippers'][-1] += ratio
                        popCityinfo['Lakers Clippers'][-1] = popCityinfo['Lakers Clippers'][-1]/2
                        LA2 = True

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    for teams in popCityinfo.items():
        population_by_region.append(int(teams[1][1]))
        win_loss_by_region.append(teams[1][2])

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    if (pci == True):
        return(popCityinfo)
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#2018 data
cleanMLB = mlb_df.iloc[0:30]

def mlb_correlation(pci=False): 
    # YOUR CODE HERE
    popCityinfo = {}

    for team in cities['MLB']:
        team2 = re.sub('\[note .*]','', team)
        if team2 != '—' and team2!='':
            cities['MLB'] = cities['MLB'].replace([team], team2)
            metropolitanArea = cities['Metropolitan area'].where(cities['MLB'] == team2).dropna()
            population = cities['Population (2016 est.)[8]'].where(cities['MLB'] == team2).dropna()
            popCityinfo[team2] = [metropolitanArea.values[0], population.values[0]]
   
    #create col in df of w:l ratios/ team
    cleanMLB['W:L'] = cleanMLB['W'].astype(int)/(cleanMLB['W'].astype(int)+cleanMLB['L'].astype(int))
    
    #two team cities
    popCityinfo['Cubs White Sox'].append((float(cleanMLB['W:L'].where(cleanMLB['team'] == 'Chicago White Sox').dropna()) +
                              float(cleanMLB['W:L'].where(cleanMLB['team'] == 'Chicago Cubs').dropna()))/2)
    
    popCityinfo['Yankees Mets'].append((float(cleanMLB['W:L'].where(cleanMLB['team'] == 'New York Yankees').dropna()) +
                            float(cleanMLB['W:L'].where(cleanMLB['team'] == 'New York Mets').dropna()))/2)
    
    popCityinfo['Dodgers Angels'].append((float(cleanMLB['W:L'].where(cleanMLB['team'] == 'Los Angeles Dodgers').dropna()) +
                            float(cleanMLB['W:L'].where(cleanMLB['team'] == 'Los Angeles Angels').dropna()))/2)
    
    popCityinfo['Giants Athletics'].append((float(cleanMLB['W:L'].where(cleanMLB['team'] == 'San Francisco Giants').dropna()) +
                               float(cleanMLB['W:L'].where(cleanMLB['team'] == 'Oakland Athletics').dropna()))/2)
    
    for key in popCityinfo:
        for t in cleanMLB['team']:
            if key in t:
                ratio = float(cleanMLB['W:L'].where(cleanMLB['team'] == t).dropna())
                popCityinfo[key].append(ratio)
                

    
    
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    for teams in popCityinfo.items():
        population_by_region.append(int(teams[1][1]))
        win_loss_by_region.append(teams[1][2])

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
        
    if (pci == True):
        return(popCityinfo)

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.1502769830266931

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nfl2018 = nfl_df[1:40]
nfl2018 = nfl2018.drop(index=[5,10,15,20,25,30,35], axis=1)


def nfl_correlation(pci=False):
    
    popCityinfo = {}
    for team in cities['NFL']:
        team2 = re.sub('\[note .*]','', team)
        if team2 != '—' and team2!='' and team2!='— ':
            cities['NFL'] = cities['NFL'].replace([team], team2)
            metropolitanArea = cities['Metropolitan area'].where(cities['NFL'] == team2).dropna()
            population = cities['Population (2016 est.)[8]'].where(cities['NFL'] == team2).dropna()
            popCityinfo[team2] = [metropolitanArea.values[0], population.values[0]]

    nfl2018['wlr'] = nfl2018['W'].astype(int)/(nfl2018['W'].astype(int)+nfl2018['L'].astype(int))

    for team in popCityinfo:
        for t in nfl2018['team']:
            if team in t:
                ratio = float(nfl2018['wlr'].where(nfl2018['team'] == t).dropna())
                popCityinfo[team].append(ratio)

    r1 = float(nfl2018['wlr'].where(nfl2018['team'] == 'New York Jets').dropna())+float(nfl2018['wlr'].where(nfl2018['team'] == 'New York Giants').dropna())
    r1 = r1/2
    popCityinfo['Giants Jets'].append(r1)

    r2 = float(nfl2018['wlr'].where(nfl2018['team'] == 'Los Angeles Rams*').dropna())+float(nfl2018['wlr'].where(nfl2018['team'] == 'Los Angeles Chargers+').dropna())
    r2 = r2/2
    popCityinfo['Rams Chargers'].append(r2)

    r3 = float(nfl2018['wlr'].where(nfl2018['team'] == 'San Francisco 49ers').dropna())+float(nfl2018['wlr'].where(nfl2018['team'] == 'Oakland Raiders').dropna())
    r3 = r3/2
    popCityinfo['49ers Raiders'].append(r3)
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    for key,value in popCityinfo.items():
        population_by_region.append(int(value[1]))
        win_loss_by_region.append(value[2])

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    
    if (pci == True):
        return(popCityinfo)

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349414

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [20]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE    
    nba_nhl = []
    nhl_nba = []
    for k,v in nba_correlation(pci=True).items():
        for k2, v2, in nhl_correlation(pci=True).items():
            if v[0] == v2[0]:
                nba_nhl.append(v[2])
                nhl_nba.append(v2[2])

    nba_nfl = []
    nfl_nba = []          
    for k,v in nba_correlation(pci=True).items():
        for k2, v2, in nfl_correlation(pci=True).items():
            if v[0] == v2[0]:
                nba_nfl.append(v[2])
                nfl_nba.append(v2[2])

    nba_mlb = []
    mlb_nba = []          
    for k,v in nba_correlation(pci=True).items():
        for k2, v2, in mlb_correlation(pci=True).items():
            if v[0] == v2[0]:
                nba_mlb.append(v[2])
                mlb_nba.append(v2[2])

    nhl_nfl =[]
    nfl_nhl = []        
    for k,v in nhl_correlation(pci=True).items():
        for k2, v2, in nfl_correlation(pci=True).items():
            if v[0] == v2[0]:
                nhl_nfl.append(v[2])
                nfl_nhl.append(v2[2])

    nhl_mlb = []
    mlb_nhl = []
    for k,v in nhl_correlation(pci=True).items():
        for k2, v2, in mlb_correlation(pci=True).items():
            if v[0] == v2[0]:
                nhl_mlb.append(v[2])
                mlb_nhl.append(v2[2])

    nfl_mlb = []
    mlb_nfl = []
    for k,v in nfl_correlation(pci=True).items():
        for k2, v2, in mlb_correlation(pci=True).items():
            if v[0] == v2[0]:
                nfl_mlb.append(v[2])
                mlb_nfl.append(v2[2])
                
    def pval(t1, t2):
        return(stats.ttest_rel(t1, t2)[1])
    nflnba = pval(nfl_nba, nba_nfl)
    nflnhl = pval(nfl_nhl, nhl_nfl)
    nflmlb = pval(nfl_mlb, mlb_nfl)
    nbanhl = pval(nba_nhl, nhl_nba)
    nbamlb = pval(nba_mlb, mlb_nba)
    nhlmlb = pval(nhl_mlb, mlb_nhl)
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = {'NFL':[np.nan, nflnba, nflnhl, nflmlb], 'NBA':[nflnba, np.nan, nbanhl, nbamlb], 'NHL':[nflnhl, nbanhl, np.nan, nhlmlb], 'MLB':[nflmlb, nbamlb, nhlmlb, np.nan]}
    p_values = pd.DataFrame({k:v for k,v in sports.items()}, index=sports.keys())
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
